In [ ]:
shakespeare_filepath = r'dataset\input.txt'

In [ ]:
with open(shakespeare_filepath, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
len(text)

In [ ]:
print(text[:1000])

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars),'\n', vocab_size)

In [ ]:
stoi = { ch: i for i,ch in enumerate(chars)}
itos = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('Aa'))
print(decode(encode('Aa')))

In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:1000])
print(data.shape)

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

print(x,y)

In [ ]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'input is {context} then target is {target}')

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

    

In [ ]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t+1]
        targets = yb[b, t]
        print(f'input is {context}, target is {targets}')
print('-------------')

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module): #Our model module, with subclass modules nested within
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #Embedding table using nn.Embedding by PyTorch of size (Vocab_size, Vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) #plucking our logits from Embedding table using index
        
        if targets is None:  #Used for genration purposes, when we are not training and no ground truth targets are fed, then the loss is not calculated
            loss=None
        else:
            B, T, C = logits.shape #Unpacking the logits into Batch, Time and Channel
            logits = logits.view(B*T, C) #Stretching our logits shape, in this case multliplying B*T, C --> (B,C) tensor as required by our loss function that expects 2 dimensions (Batch, Channel)
            targets = targets.view(B*T) #Stretching our targets similar to our logits, creating one dimension of B*T --> (B)
            loss = F.cross_entropy(logits, targets) #Calculatin our loss, logits (B,C) against our targets (B)
            
        return logits , loss
        
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[ : , -1, : ]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
            
        return idx

m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

output_tokens = m.generate(torch.zeros((1,1) , dtype = torch.long), max_new_tokens = 100)
print(decode(output_tokens[0].tolist()))

In [ ]:
import torch.optim
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100000):
    
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [ ]:
output_tokens = m.generate(torch.zeros((1,1) , dtype = torch.long), max_new_tokens = 1000)
print(decode(output_tokens[0].tolist()))